In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, ReLU, Conv2D

from tensorflow.nn import conv2d, conv2d_transpose
from tensorflow.io import read_file
from tensorflow.image import decode_image
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sys
from pathlib import Path
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Helpers

In [2]:
def _conv2d(x, kernel):
    return conv2d(x, kernel, strides=[1, 2, 2, 1], padding='SAME')

def _conv2d_transpose(x, kernel, output_shape):
    return conv2d_transpose(
            x, kernel,
            output_shape=output_shape,
            strides=[1, 2, 2, 1],
            padding='SAME')

def load_image(image_path):
    image = read_file(image_path)
    image = decode_image(image)

    return image

In [3]:
class WaveletPooling(Layer):
    """
    Wavelet Pooing Custom Layer
    """
    def __init__(self, name=''):
        super(WaveletPooling, self).__init__()
        self._name = name
        square_of_2 = tf.math.sqrt(tf.constant(2, dtype=tf.float32))
        L = tf.math.divide(
            tf.constant(1, dtype=tf.float32),
            tf.math.multiply(square_of_2, tf.constant([[1, 1]], dtype=tf.float32))
        )
        H = tf.math.divide(
            tf.constant(1, dtype=tf.float32),
            tf.math.multiply(square_of_2, tf.constant([[-1, 1]], dtype=tf.float32))
        )

        self.LL = tf.reshape(tf.math.multiply(tf.transpose(L), L), (1, 2, 2, 1))
        self.LH = tf.reshape(tf.math.multiply(tf.transpose(L), H), (1, 2, 2, 1))
        self.HL = tf.reshape(tf.math.multiply(tf.transpose(H), L), (1, 2, 2, 1))
        self.HH = tf.reshape(tf.math.multiply(tf.transpose(H), H), (1, 2, 2, 1))

    def call(self, inputs):
        LL, LH, HL, HH = self.repeat_filters(inputs.shape[-1])
        return [_conv2d(inputs, LL),
                _conv2d(inputs, LH),
                _conv2d(inputs, HL),
                _conv2d(inputs, HH)]

    def compute_output_shape(self, input_shape):
        shape = (
            input_shape[0], input_shape[1] // 2,
            input_shape[2] // 2, input_shape[3]
        )

        return [shape, shape, shape, shape]

    def repeat_filters(self, repeats):
        # Can we optimize this?
        return [
            tf.transpose(tf.repeat(self.LL, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.LH, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.HL, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.HH, repeats, axis=0), (1, 2, 3, 0))
        ]

class WaveletUnpooling(Layer):
    """
    Wavelet Unpooing Custom Layer
    """
    def __init__(self, name):
        super(WaveletUnpooling, self).__init__()
        self._name = name
        square_of_2 = tf.math.sqrt(tf.constant(2, dtype=tf.float32))
        L = tf.math.divide(
            tf.constant(1, dtype=tf.float32),
            tf.math.multiply(square_of_2, tf.constant([[1, 1]], dtype=tf.float32))
        )
        H = tf.math.divide(
            tf.constant(1, dtype=tf.float32),
            tf.math.multiply(square_of_2, tf.constant([[-1, 1]], dtype=tf.float32))
        )

        self.LL = tf.reshape(tf.math.multiply(tf.transpose(L), L), (1, 2, 2, 1))
        self.LH = tf.reshape(tf.math.multiply(tf.transpose(L), H), (1, 2, 2, 1))
        self.HL = tf.reshape(tf.math.multiply(tf.transpose(H), L), (1, 2, 2, 1))
        self.HH = tf.reshape(tf.math.multiply(tf.transpose(H), H), (1, 2, 2, 1))

    def call(self, inputs):
        LL_in, LH_in, HL_in, HH_in, tensor_in = inputs
        LL, LH, HL, HH = self.repeat_filters(LL_in.shape[-1])
        out_shape = tf.shape(tensor_in)

        return _conv2d_transpose(LL_in, LL, output_shape=out_shape) + _conv2d_transpose(LH_in, LH, output_shape=out_shape) + _conv2d_transpose(HL_in, HL, output_shape=out_shape) + _conv2d_transpose(HH_in, HH, output_shape=out_shape)

    def compute_output_shape(self, input_shape):
        _ip_shape = input_shape[0]
        shape = (
            _ip_shape[0],
            _ip_shape[1] * 2,
            _ip_shape[2] * 2,
            sum(ips[3] for ips in input_shape)
        )

        return shape

    def repeat_filters(self, repeats):
        # Can we optimize this?
        return [
            tf.transpose(tf.repeat(self.LL, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.LH, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.HL, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.HH, repeats, axis=0), (1, 2, 3, 0)),
        ]

class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [tf.keras.layers.InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        if s[1] == None:
            return (None, None, None, s[3])
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad, h_pad = self.padding
        return tf.pad(x, [[0, 0], [h_pad, h_pad], [w_pad, w_pad], [0, 0]], 'REFLECT')

    def get_config(self):
        config = super(ReflectionPadding2D, self).get_config()
        print(config)
        return config

class CNNBlock(Layer):
    def __init__(self, filters, kernel, name):
        super(CNNBlock, self).__init__()
        self._name = name
        
        self.padding = ReflectionPadding2D()
        self.conv2d = Conv2D(filters, kernel, padding = 'valid')
        self.activation = ReLU()

    def call(self, inputs):
        x = self.padding(inputs)
        x = self.conv2d(x)
        x = self.activation(x)
        
        return x

In [4]:
def preprocess_feat(feat, center=False):
    feat = tf.reshape(feat, (-1, feat.shape[-1]))
    feat = tf.transpose(feat)

    feat_mean_raw = tf.math.reduce_mean(feat, 1)

    if center:
        feat_mean = tf.expand_dims(feat_mean_raw, 1)
        feat = tf.subtract(feat, feat_mean)

    return feat, feat_mean_raw

def center_feat(feat):
    feat_mean = tf.math.reduce_mean(feat, 1)
    feat_mean = tf.expand_dims(feat_mean, 1)
    return tf.subtract(feat, feat_mean)

def get_svd(feat):
    feat = center_feat(feat)
    feat = tf.matmul(feat, feat, transpose_b=True) / (feat.shape[1] - 1)
    feat = feat + tf.eye(feat.shape[0])
    
    return tf.linalg.svd(feat)

def get_style_correlation_transform(feat, return_mean=False, normalize=False):
    feat, mean = preprocess_feat(feat)
    s_e, _, s_v = get_svd(feat)
    # The inverse of the content singular values matrix operation (^-0.5)
    s_e = tf.pow(s_e, 0.5)

    EDE = tf.matmul(tf.matmul(s_v, tf.linalg.diag(s_e)), s_v, transpose_b=True)

    if normalize:
        EDE = EDE / tf.reduce_max(tf.abs(EDE))

    return (EDE, mean) if return_mean else EDE

def wct(content_feat_raw, style_feat_raw, alpha = 1):
    style_EDE, style_mean = get_style_correlation_transform(style_feat_raw, return_mean = True)
    content_feat, content_mean = preprocess_feat(content_feat_raw, center=True)
    
    c_e, _, c_v = get_svd(content_feat)
    c_e = tf.pow(c_e, -0.5)
    
    content_EDE = tf.matmul(tf.matmul(c_v, tf.linalg.diag(c_e)), c_v, transpose_b=True)
    content_whitened = tf.matmul(content_EDE, content_feat)

    final_out = tf.matmul(style_EDE, content_whitened)
    final_out = tf.add(final_out, tf.expand_dims(style_mean, 1))
    final_out = tf.clip_by_value(
        final_out,
        tf.math.reduce_min(content_feat),
        tf.math.reduce_max(content_feat),
    )
    
    final_out = tf.reshape(tf.transpose(final_out), content_feat_raw.shape)
    final_out = alpha * final_out + (1 - alpha) * content_feat_raw
    return final_out

r = wavelet_ae.transfer(tf.expand_dims(test_img, 0), tf.expand_dims(test_img_style, 0), skips_transfer=False, decoder_transfer=False)[0]
plt.imshow(r)

NameError: name 'wavelet_ae' is not defined

In [5]:
class WaveletEncoder(Model):
    def __init__(self):
        super().__init__()
        self._name = 'WaveletEncoder'
        
        self.preprocessing_conv2d = Conv2D(3, 1, padding = 'valid')
        
        ### Block 1
        self.block1_conv2d_1 = CNNBlock(64, 3, 'WE_block1_conv2d_1')
        self.block1_conv2d_2 = CNNBlock(64, 3, 'WE_block1_conv2d_2')
        self.block1_pooling = WaveletPooling('WE_block1_pooling')
        
        ### Block 2
        self.block2_conv2d_1 = CNNBlock(128, 3, 'WE_block2_conv2d_1')
        self.block2_conv2d_2 = CNNBlock(128, 3, 'WE_block2_conv2d_2')
        self.block2_pooling = WaveletPooling('WE_block2_pooling')
        
        ### Block 3
        self.block3_conv2d_1 = CNNBlock(256, 3, 'WE_block3_conv2d_1')
        self.block3_conv2d_2 = CNNBlock(256, 3, 'WE_block3_conv2d_2')
        self.block3_conv2d_3 = CNNBlock(256, 3, 'WE_block3_conv2d_3')
        self.block3_conv2d_4 = CNNBlock(256, 3, 'WE_block3_conv2d_4')
        self.block3_pooling = WaveletPooling('WE_block3_pooling')
        
        ### Block 4
        self.block4_conv2d_1 = CNNBlock(512, 3, 'WE_block4_conv2d_1')

    def call(self, inputs, style_feat = None, trainable = False):
        wavelet_skips = {
            'block1': None,
            'block2': None,
            'block3': None,
        }
        
        features = {
            'block1': None,
            'block2': None,
            'block3': None
        }
        
        x = self.preprocessing_conv2d(inputs)
        
        x = self.block1_conv2d_1(x)
        x = self.block1_conv2d_2(x)
        LL_1, LH_1, HL_1, HH_1 = self.block1_pooling(x)
        wavelet_skips['block1'] = [LH_1, HL_1, HH_1, x]
        features['block1'] = LL_1
        
        if style_feat:
            LL_1 = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (LL_1, style_feat['block1']),
                dtype=LL_1.dtype
            )
            
        x = self.block2_conv2d_1(LL_1)
        x = self.block2_conv2d_2(x)
        LL_2, LH_2, HL_2, HH_2 = self.block2_pooling(x)
        wavelet_skips['block2'] = [LH_2, HL_2, HH_2, x]
        features['block2'] = LL_2
        
        if style_feat:
            LL_2 = tf.map_fn(
                lambda y: wct(y[0], y[1]),
                (LL_2, style_feat['block2']),
                dtype=LL_2.dtype
            )
            
        x = self.block3_conv2d_1(LL_2)
        x = self.block3_conv2d_2(x)
        x = self.block3_conv2d_3(x)
        x = self.block3_conv2d_4(x)
        LL_3, LH_3, HL_3, HH_3 = self.block3_pooling(x)
        wavelet_skips['block3'] = [LH_3, HL_3, HH_3, x]
        features['block3'] = LL_3
        
        if style_feat:
            LL_3 = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (LL_3, style_feat['block3']),
                dtype=LL_3.dtype
            )
        
        x = self.block4_conv2d_1(LL_3)
        features['block4'] = x
        
        if style_feat:
            x = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (x, style_feat['block4']),
                dtype=x.dtype
            )
            
        return x, wavelet_skips, features
    
class WaveletDecoder(Model):
    def __init__(self):
        super().__init__()
        self._name = 'WaveletDecoder'

        ### Block 3
        self.block3_conv2d_1 = CNNBlock(256, 3, 'WD_block4_conv2d_1')
        self.block3_unpooling = WaveletUnpooling('WD_block4_unpooling')
        self.block3_conv2d_2 = CNNBlock(256, 3, 'WD_block3_conv2d_2')
        self.block3_conv2d_3 = CNNBlock(256, 3, 'WD_block3_conv2d_3')
        self.block3_conv2d_4 = CNNBlock(256, 3, 'WD_block3_conv2d_4')

        ### Block 2
        self.block2_conv2d_1 = CNNBlock(128, 3, 'WE_block2_conv2d_1')
        self.block2_unpooling = WaveletUnpooling('WE_block2_unpooling')
        self.block2_conv2d_2 = CNNBlock(128, 3, 'WE_block2_conv2d_2')

        ### Block 1
        self.block1_conv2d_1 = CNNBlock(64, 3, 'WE_block1_conv2d_1')
        self.block1_unpooling = WaveletUnpooling('WE_block1_unpooling')
        self.block1_conv2d_2 = CNNBlock(64, 3, 'WE_block1_conv2d_2')

        self.post_processing_padding = ReflectionPadding2D()
        self.post_processing_conv2d = Conv2D(3, 3, padding = 'valid')

    def call(self, inputs, skips, style_feat = None, trainable = False):
        features = {
            'block3': None,
            'block2': None,
            'block1': None,
        }
        
        x = self.block3_conv2d_1(inputs)
        x = self.block3_unpooling([x, *skips['block3']])
        x = self.block3_conv2d_2(x)
        x = self.block3_conv2d_3(x)
        x = self.block3_conv2d_4(x)
        features['block3'] = x
        
        if style_feat:
            x = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (x, style_feat['block3']),
                dtype=x.dtype
            )
        
        x = self.block2_conv2d_1(x)
        x = self.block2_unpooling([x, *skips['block2']])
        x = self.block2_conv2d_2(x)
        features['block2'] = x

        if style_feat:
            x = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (x, style_feat['block2']),
                dtype=x.dtype
            )
            
        x = self.block1_conv2d_1(x)
        x = self.block1_unpooling([x, *skips['block1']])
        x = self.block1_conv2d_2(x)
        features['block1'] = x
        
        if style_feat:
            x = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (x, style_feat['block1']),
                dtype=x.dtype
            )
            
        x = self.post_processing_padding(x)
        x = self.post_processing_conv2d(x)

        return x, features

In [6]:
class WaveletAE(Model):
    def __init__(self):
        super().__init__()
        self._name = 'WaveletAE'
        self.encoder = WaveletEncoder()
        self.decoder = WaveletDecoder()
        self.load_weights('../weights/wavelet_autoencoder')

    def call(self, inputs, trainable = False):
        x, skips, _ = self.encoder(inputs)
        output, _ = self.decoder(x, skips)

        return output
        
    def transfer(self, content_img, style_img, encoder_transfer = True, skips_transfer = True, decoder_transfer = True, alpha = 1):
        style_features, style_skips = self.get_features(style_img)
        x, content_skips, _ = self.encoder(content_img, style_features['encoder'] if encoder_transfer else None)
        
        if skips_transfer:
            for key in content_skips.keys():
                for i in range(3):
                    content_skips[key][0] = tf.map_fn(
                        lambda x: wct(x[0], x[1]),
                        (content_skips[key][0], style_skips[key][0]),
                        dtype=tf.float32
                    )
                
        out, _ = self.decoder(x, content_skips, style_features['decoder'] if decoder_transfer else None)
        out = tf.clip_by_value(out, 0, 1)
        
        return out
    
    def get_features(self, inputs):
        encoder_out, skips, encoder_feat = self.encoder(inputs)
        _, decoder_feat = self.decoder(encoder_out, skips)
        
        features = {
            'encoder': encoder_feat,
            'decoder': decoder_feat,
        }
        
        return features, skips
    
    def get_style_correlations(self, inputs, blocks=['block1', 'block2', 'block3', 'block4'], ede=True, normalize=True):
        _, _, encoder_feat = self.encoder(inputs)
        correlations = []
        means = []

        def process_correlation(feature_map, normalize=normalize):
            feat, _ = preprocess_feat(feature_map, center=True)
            feat = tf.matmul(feat, feat, transpose_b=True) / (feat.shape[1] - 1)

            if normalize:
                feat = feat / tf.reduce_max(tf.abs(feat))

            return feat

        def process_feat(feat):
            if ede:
                return get_style_correlation_transform(feat, normalize=normalize)
            else:
                return process_correlation(feat)

        for block in blocks:
            corr = tf.map_fn(process_feat, encoder_feat[block])
            mean = tf.map_fn(lambda feat: preprocess_feat(feat, center=False)[1], encoder_feat[block])
            correlations.append(corr)
            means.append(mean)

        return correlations, means
        
test_img = tf.io.read_file('../assets/moli_content.jpg')
test_img = tf.image.decode_image(test_img, dtype=tf.float16)

test_img_style = tf.io.read_file('../assets/moli_style.jpg')
test_img_style = tf.image.decode_image(test_img_style, dtype=tf.float16)
test_img_style = tf.image.resize(test_img_style, (test_img_style.shape[0] // 4, test_img_style.shape[1] // 4))

wavelet_ae = WaveletAE()
# r = wavelet_ae.transfer(tf.expand_dims(test_img, 0), tf.expand_dims(test_img_style, 0), skips_transfer=False, decoder_transfer=False)[0]
# plt.imshow(r)
wavelet_ae.get_style_correlations(tf.expand_dims(test_img, 0))[1]

[<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
 array([[2.0240002 , 1.6731993 , 1.1463751 , 0.92737335, 1.4541161 ,
         1.7794412 , 1.7465117 , 0.71380115, 1.8442959 , 1.645102  ,
         2.013398  , 2.1432931 , 1.6043837 , 1.7198778 , 1.9763225 ,
         1.2551997 , 1.8265654 , 1.8353379 , 1.9723362 , 1.5268104 ,
         2.0193257 , 1.688461  , 1.8234606 , 1.5682324 , 0.5892709 ,
         2.0896099 , 1.761323  , 0.6083652 , 1.7204839 , 1.7739407 ,
         1.9185908 , 2.153166  , 1.6460828 , 1.59885   , 1.9544923 ,
         1.9103466 , 1.4840791 , 1.9276464 , 1.5477422 , 1.8149924 ,
         1.7897822 , 1.621694  , 1.5202886 , 1.9548333 , 1.6892569 ,
         2.186989  , 1.6573479 , 1.9315081 , 1.7060064 , 1.8213422 ,
         1.9746875 , 1.6584809 , 1.3480985 , 1.8407003 , 1.947064  ,
         1.8487458 , 1.8138773 , 1.9131799 , 1.7322302 , 1.947852  ,
         1.4328079 , 2.139583  , 1.8707486 , 1.8873596 ]], dtype=float32)>,
 <tf.Tensor: shape=(1, 128), dtype=float32, nu

In [77]:
def sample_from_corr_matrix(sigma, num_features = None, eigenvalues = None, eigenvectors = None):
    data = tf.eye(sigma.shape[0], num_features)

    if not eigenvectors or not eigenvectors:
        eigenvalues, u, eigenvectors = tf.linalg.svd(sigma)
        eigenvalues = tf.linalg.diag(eigenvalues)
        
    data = tf.matmul(tf.matmul(eigenvectors, tf.sqrt(eigenvalues)), data)
    
    return data

In [ ]:
feature_description = {
    'resized_image': tf.io.FixedLenFeature([], tf.string),
    'block1': tf.io.FixedLenFeature([], tf.string),
    'block2': tf.io.FixedLenFeature([], tf.string),
    'block3': tf.io.FixedLenFeature([], tf.string),
    'block4': tf.io.FixedLenFeature([], tf.string),

    'music_spec': tf.io.FixedLenFeature([], tf.string),
}

def _parse_function(data):
    parsed_data = tf.io.parse_single_example(data, feature_description)
    
    resized_image = tf.image.decode_jpeg(parsed_data["resized_image"], channels = 3)
    block1 = tf.io.parse_tensor(parsed_data["block1"], tf.float16)
    block2 = tf.io.parse_tensor(parsed_data["block2"], tf.float16)
    block3 = tf.io.parse_tensor(parsed_data["block3"], tf.float16)
    block4 = tf.io.parse_tensor(parsed_data["block4"], tf.float16)
    
    music_spec = tf.io.parse_tensor(parsed_data["music_spec"], tf.float16)
    
    return resized_image

In [ ]:
training_data_dir = Path(os.getcwd()).parent  / "data" / "tfrecords" / "train"

train_ds_files = tf.data.Dataset.list_files(str(training_data_dir / '*'), shuffle=True, seed=4321)

AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 16

train_ds = tf.data.TFRecordDataset(train_ds_files, compression_type="GZIP")
train_ds = train_ds.shuffle(1024)
train_ds = train_ds.map(_parse_function, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.apply(tf.data.experimental.ignore_errors())
train_ds = train_ds.repeat()
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

# val_ds = tf.data.TFRecordDataset(str(val_data_path), compression_type="GZIP")
# val_ds = val_ds.map(_parse_function, num_parallel_calls=AUTOTUNE)
# val_ds = val_ds.batch(BATCH_SIZE)
# val_ds = val_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)
# val_ds = val_ds.apply(tf.data.experimental.ignore_errors())
# val_ds = val_ds.repeat()
# val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# test_ds = tf.data.TFRecordDataset(str(test_data_path), compression_type="GZIP")
# test_ds = test_ds.map(_parse_function, num_parallel_calls=AUTOTUNE)
# test_ds = test_ds.batch(BATCH_SIZE)
# test_ds = test_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)
# test_ds = test_ds.apply(tf.data.experimental.ignore_errors())
# test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
for i in train_ds.take(5):
    print(i.shape)